In [35]:
import math
import numpy as np



class ModelSets:
    def __init__(self, Psi, Psi1, Psi2, Phi, I, O, A_in, A_out, A, s_size=3):
        # Posições dentro do armazém
        self.Psi = Psi
        # Posições no nível inferior
        self.Psi1 = Psi1
        # Posições no nível superior
        self.Psi2 = Psi2
        # Todas as posições, incluindo entrada e saída
        self.Phi = Phi
        # Posições de entrada
        self.I = I
        # Posições de saída
        self.O = O
        # lista de bobinas que entram no armazém
        self.A_in = A_in
        # lista de bobinas que saem do armazém
        self.A_out = A_out
        # lista de todas as bobinas
        self.A = A
        # Intervalos de tempo para movimentação
        S = [range(s_size)]  # Intervalos de tempo para movimentação
        self.S = S  # Conjunto seções de tempos

    def get_sets_sizes(self):
        return {
            "Psi": len(self.Psi),
            "Psi1": len(self.Psi1),
            "Psi2": len(self.Psi2),
            "Phi": len(self.Phi),
            "I": len(self.I),
            "O": len(self.O),
            "A_in": len(self.A_in),
            "A_out": len(self.A_out),
            "A": len(self.A),
            "S": len(self.S),
        }

    def set_range(self, set_name):
        if hasattr(self, set_name):
            return range(len(getattr(self, set_name)))
        else:
            raise ValueError(f"Set '{set_name}' not found in ModelSets.")

def gerar_subconjunto(conjunto: range, tamanho: int) -> list:
    """Gera um subconjunto aleatório de um conjunto dado."""

    if tamanho > len(conjunto):
        raise ValueError("O tamanho do subconjunto não pode ser maior que o conjunto original.")
    subconj = []
    while len(subconj) < tamanho:
        elemento = np.random.choice(conjunto)
        if elemento not in subconj:
            subconj.append(elemento)
    return subconj

def gerar_posicoes(
    num_fileiras: int,
    num_posicoes_nivel_inferior: int,
    num_entrada_saida: int,
    num_bobinas: int = 3,
    num_bobinas_entrada: int = 1,
    num_bobinas_saida: int = 1,
):
    """
    Gera as posições de armazenamento. No formato proposto por 
    Weckenborg et al. (2025).
    Exemplo para uma pilha de 3 bobinas

    I + [(111), (112), (121)] + O
    """
    posicoes = []
    for y in range(num_fileiras):
        for x in range(num_posicoes_nivel_inferior):
            print("x", x)
            posicoes.append((y + 1, 1, x + 1))  # nível inferior
            if (x != num_posicoes_nivel_inferior):
                posicoes.append((y + 1, 2, x + 1))  # nível superior, entre duas inferiores

    I = [(-1, 1, y) for y in range(num_entrada_saida)]
    O = [(num_fileiras + 1, 1, y) for y in range(num_entrada_saida)]

    A = range(num_bobinas)

    # A_in é uma lista aleatória de subitens de A.
    A_in = gerar_subconjunto(A, num_bobinas_entrada)
    A_out = gerar_subconjunto(A, num_bobinas_saida)

    # Cria model sets
    model_sets = ModelSets(
        Psi=posicoes,
        Psi1=posicoes[0::2],
        Psi2=posicoes[1::2],
        Phi=I + posicoes + O,
        I=I,
        O=O,
        A_in=A_in,  # Lista de bobinas que entram no armazém
        A_out=A_out,  # Lista de bobinas que saem do armazém
        A=A,  # Lista de todas as bobinas
        s_size=num_bobinas * 3
    )

    return model_sets


def places_are_equal(place1, place2) -> bool:
    return place1[0] == place2[0] and place1[1] == place2[1] and place1[2] == place2[2]

class ModelParameters:
    def __init__(self, t_load: np.ndarray, t_empty: np.ndarray, E_load: np.ndarray, E_empty: np.ndarray):
        self.t_load = t_load
        self.t_empty = t_empty
        self.E_load = E_load
        self.E_empty = E_empty

    def get_maximum_time(self):
        """"
        Calcula o tempo máximo de movimentação baseado nos tempos de carregamento e esvaziamento.
        
        - O tempo máximio é útil para definir os limites de tempo para entrada e saída das bobinas.
        - O tempo máximo é o maior valor entre os tempos de carregamento e esvaziamento.
        """
        # Retorna o tempo máximo de movimentação
        return max(np.max(self.t_load), np.max(self.t_empty))


def gerar_custos_de_movimentacao(Phy: list[tuple], A_size: int):
    """
    Gera os custos de movimentação para o problema de armazenamento.
    
    Cada movimentação vertical no diametro de uma bobina custa 1 unidade de tempo.
    Cada movimentação horizontal no diametro de uma bobina custa 0.5 unidades de tempo.
    
    """
    t_load = np.zeros((len(Phy), len(Phy)), dtype=float)
    t_empty = np.zeros((len(Phy), len(Phy)), dtype=float)
    E_load = np.zeros((len(Phy), len(Phy), A_size), dtype=float)
    E_empty = np.zeros((len(Phy), len(Phy)), dtype=float)

    for idk, k in enumerate(Phy):
        for idq, q in enumerate(Phy):
            if idk == idq:
                continue
            t_vertical = 3 - k[1] +  3 - q[1]
            t_horizontal = ((q[0] - k[0]) + (q[2] - k[2])) *0.5/math.sqrt(2)
            ## TODO: Avaliar pesos abaixo
            t_load[idk][idq] = t_vertical * 20 + t_horizontal * 10
            t_empty[idk][idq] = t_load[idk][idq] * 0.9

    for idk, k in enumerate(Phy):
        for idq, q in enumerate(Phy):
            E_empty[idk][idq] = t_empty[idk][idq] * 20
            for a in range(A_size):
                # Por simplicidade, todos os custos de movimentação são iguais
                # Isso pode ser ajustado para refletir a realidade de bobinas difrentes
                E_load[idk][idq][a] = t_load[idk][idq] * 100

    # Cria uma instância de ModelParameters com os tempos e custos calculados

    return {
        "t_load": t_load,
        "t_empty": t_empty,
        "E_load": E_load,
        "E_empty": E_empty
    }



In [ ]:
from gurobipy import Model, GRB, quicksum


class ModelPriorities:
    def __init__(self, A_in_size=1, A_out_size=1, s_size=3, maximum_time=3600):
        # Prioridades para as bobinas, por exemplo, s_size = 3 para três bobinas
        self.A_in_size = A_in_size
        self.A_out_size = A_out_size
        self.s_size = s_size
        self.maximum_time = maximum_time

    def generate_priorities(self):
        """
        Gera prioridades para as bobinas de entrada e saída.
        As prioridades são a hora e saída maximas e mínimas para cada bobina.
        O calculo é baseado no tempo máximo de movimentação e ele tem que ser
        maior que o tempo de movimentação.
        """
        rnd_wait_time = 3 * self.maximum_time

        sigma_minus = [i * self.maximum_time for i in range(self.A_in_size)]
        sigma_plus = [
            (i + 1) * self.maximum_time for i in range(self.A_in_size)
        ]

        omega_minus = [
            (i + 2) * (self.maximum_time + rnd_wait_time)
            for i in range(self.A_out_size)
        ]
        omega_plus = [
            (i + 3) * (self.maximum_time + 2 * rnd_wait_time)
            for i in range(self.A_out_size)
        ]
        return {
            "sigma_minus": sigma_minus,
            "sigma_plus": sigma_plus,
            "omega_minus": omega_minus,
            "omega_plus": omega_plus,
        }

    def get_min_size(self):
        return min(self.A_in_size, self.A_out_size) * 2


def gerar_modelo(
    model_sets: ModelSets,
    priorities: ModelPriorities,
    time_limit: int = 3600,  # Tempo limite em segundos
    verbose: bool = True,
    run_id: str = "default_run_id",
) -> Model:
    model = Model(f"Armazenagem_Bobinas_{run_id}")
    model_sets_sizes = model_sets.get_sets_sizes()
    print("Tamanho dos conjuntos do modelo:", model_sets_sizes)

    M = 99999
    # Gerando prioridades

    model_priorities = priorities.generate_priorities()
    
    sigma_minus = model_priorities["sigma_minus"]
    sigma_plus = model_priorities["sigma_plus"]
    omega_minus = model_priorities["omega_minus"]
    omega_plus = model_priorities["omega_plus"]


    # Definindo variáveis de decisão
    W = model.addVars(
        len(model_sets.S),
        len(model_sets.Phi),
        len(model_sets.Phi),
        len(model_sets.A),
        vtype=GRB.BINARY,
        name="W",
    )

    V = model.addVars(
        len(model_sets.S),
        len(model_sets.Phi),
        len(model_sets.Phi),
        vtype=GRB.BINARY,
        name="V",
    )

    x = model.addVars(
        len(model_sets.S),
        len(model_sets.Phi),
        len(model_sets.A),
        vtype=GRB.BINARY,
        name="x",
    )
    tau = model.addVars(len(model_sets.S), vtype=GRB.CONTINUOUS, name="tau")

    custos = gerar_custos_de_movimentacao(model_sets.Phi, len(model_sets.A))

    # Definindo a função objetivo
    model.setObjective(
        quicksum(
            W[s, k, q, a] * custos["E_load"][k][q][a]
            for s in model_sets.set_range("S")
            for k in model_sets.set_range("Phi")
            for q in model_sets.set_range("Phi")
            for a in model_sets.set_range("A")
        )
        + quicksum(
            V[s, k, q] * custos["E_empty"][k][q]
            for s in model_sets.set_range("S")
            for k in model_sets.set_range("Phi")
            for q in model_sets.set_range("Phi")
        ),
        GRB.MINIMIZE,
    )

    # Definindo restrições -----------------------------------------------------

    # R (1) - tempo_inicial_zero (τ¹ = 0)
    model.addConstr(
        tau[0] == 0, name="R1_tempo_inicial_zero"
    )  # restrição τ¹ = 0

    # R (2) - entrada_unica_bobina
    for a in model_sets.set_range("A_in"):
        model.addConstr(
            quicksum(
                W[s, 0, q, a]
                for s in model_sets.set_range("S")
                for q in model_sets.set_range("Phi")
                if q not in model_sets.set_range("I")
            )
            == 1,
            name=f"R2_entrada_unica_bobina_{a}",
        )

    # R (3) - saida_unica_bobina
    for a in model_sets.set_range("A_out"):
        model.addConstr(
            quicksum(
                W[s, k, model_sets_sizes["Phi"] - 1, a]
                for s in model_sets.set_range("S")
                for k in model_sets.set_range("Phi")
                if k not in model_sets.set_range("O")
            )
            == 1,
            name=f"R3_saida_unica_bobina_{a}",
        )

    # R (4) - nao_entrega_bobina_armazenada
    for a in model_sets.set_range("A"):
        if a not in model_sets.set_range("A_out"):
            model.addConstr(
                quicksum(
                    W[s, k, len(model_sets.Phi) - 1, a]
                    for s in model_sets.set_range("S")
                    for k in model_sets.set_range("Phi")
                )
                == 0,
                name=f"R4_nao_entrega_bobina_{a}",
            )
    
    # # R (5) - janela_max_entrada
    for a in model_sets.set_range("A_in"):
        for s in model_sets.set_range("S"):
            model.addConstr(
                tau[s] + sigma_plus[a] <= (1 - x[s, 0, a]) * M,
                name=f"R5_janela_max_entrada_a{a}_s{s}",
            )

        # R (6) - janela_min_entrada
    for a in model_sets.set_range("A_in"):
        for s in model_sets.set_range("S"):
            model.addConstr(
                omega_minus[a] - tau[s] <= x[s, 0, a] * M,
                name=f"R6_janela_min_entrada_a{a}_s{s}",
            )

        # R (7) - tempo_min_saida
    for a in model_sets.set_range("A_out"):
        for s in model_sets.set_range("S"):
            model.addConstr(
                omega_minus[a]
                - (
                    tau[s]
                    + quicksum(
                        W[s, k, len(model_sets.Phi) - 1, a] * custos["t_load"][k, len(model_sets.Phi) - 1]
                        for k in model_sets.set_range("Phi")
                    )
                )
                <= (1 - x[s, len(model_sets.Phi) - 1, a]) * M,
                name=f"R7_tempo_min_saida_a{a}_s{s}",
            )



    # Definindo limite de tempo
    model.setParam(GRB.Param.TimeLimit, time_limit)

    if verbose:
        model.params.LogToConsole = 1

    model.update()
    return model

In [37]:

testes = [
    {
        "num_fileiras": 1,
        "num_posicoes_nivel_inferior": 2,
        "num_entrada_saida": 1,
        "num_bobinas": 1,
        "num_bobinas_entrada": 1,
        "num_bobinas_saida": 1,
    },
    {
        "num_fileiras": 1,
        "num_posicoes_nivel_inferior": 2,
        "num_entrada_saida": 1,
        "num_bobinas": 2,
        "num_bobinas_entrada": 1,
        "num_bobinas_saida": 1,
    }
]


def test():
    for teste in testes:
        sets = gerar_posicoes(
            teste["num_fileiras"],
            teste["num_posicoes_nivel_inferior"],
            teste["num_entrada_saida"],
        )

        params = gerar_custos_de_movimentacao(
            sets.Phi, teste["num_bobinas"]
        )

        params_class = ModelParameters(
            t_load=params["t_load"],
            t_empty=params["t_empty"],
            E_load=params["E_load"],
            E_empty=params["E_empty"],
        )

        priorities_generator = ModelPriorities(
            A_in_size=teste["num_bobinas_entrada"],
            A_out_size=teste["num_bobinas_saida"],
            s_size=teste["num_bobinas"],
            maximum_time=params_class.get_maximum_time(),
        )

        model = gerar_modelo(
            sets,
            priorities=priorities_generator,
            time_limit=3600,
            verbose=True,
            run_id="test_run",
        )

        model.optimize()
        if model.status == GRB.OPTIMAL:
            print("Modelo otimizado com sucesso!")
        else:
            print("Otimização falhou ou não encontrou solução ótima.")
            print("Status do modelo:", model.status)

        # print("Status:", model.printStats())
        # print("Objetivo:", model.objVal)
        # print("Tempo de execução:", model.runtime)


test()



x 0
x 1
Tamanho dos conjuntos do modelo: {'Psi': 4, 'Psi1': 2, 'Psi2': 2, 'Phi': 6, 'I': 1, 'O': 1, 'A_in': 1, 'A_out': 1, 'A': 3, 'S': 1}
Set parameter TimeLimit to value 3600
Set parameter LogToConsole to value 1
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
TimeLimit  3600

Academic license 2673907 - for non-commercial use only - registered to js___@ufmg.br
Optimize a model with 8 rows, 163 columns and 34 nonzeros
Model fingerprint: 0x51d1b2d8
Variable types: 1 continuous, 162 integer (162 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [7e+02, 9e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Presolve removed 2 rows and 146 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 10 available